In [2]:
%matplotlib inline
!pip install adjustText
import zipfile
import re
import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE
from adjustText import adjust_text

2022-09-27 16:53:29.580460: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-27 16:53:29.580486: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
from os import listdir
from operator import itemgetter
all_files=listdir('/home/slisowski/Portfolio/wig20_sentiment/translated_files_prod/embed/')

#We have to sort files by index number, but this is string so first we need to extract index as integer
file_idx=[]
for file in all_files:
    idx=file.split('_')[2]
    file_idx.append(int(idx))
file_idx

files_idx=zip(all_files, file_idx)
sorted_tuples=sorted(tuple(files_idx),key=itemgetter(1))
sorted_list_files=[]
for tup in sorted_tuples:
    dir_path='/home/slisowski/Portfolio/wig20_sentiment/translated_files_prod/embed/'
    file_path=dir_path+tup[0]
    sorted_list_files.append(file_path)
    

    
news_df=pd.concat(map(pd.read_csv, sorted_list_files), ignore_index=True)

In [5]:
tweets_df=pd.read_csv('/home/slisowski/Portfolio/tweets_news_sentiment/tweets_labelled.csv', sep=";")

In [6]:
news_headlines_df=pd.read_csv('/home/slisowski/Portfolio/tweets_news_sentiment/all-data.csv', encoding="Windows-1252", names=["sentiment","text"])

In [8]:
cbow_corpus=pd.concat([news_df['news_text'], tweets_df['text'], news_headlines_df['text']])

In [11]:
import unicodedata
from nltk.stem import WordNetLemmatizer
import re
import preprocessor as p
from bs4 import BeautifulSoup
import demoji
import contractions
from nltk.tokenize import word_tokenize


def clean_news_cbow(text):
    
    #remove emoji
    text=demoji.replace(text,' ')
    # use tweeter preprocessor to clean news
    text=p.clean(text)
    #remove html tags
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    text=contractions.fix(text)
    
    
    #remove accented characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
   
   
    #remove numbers
    text = re.sub("[^a-zA-Z]", " ", text)
    #tokenize text
    text=text.lower()
    
    list_of_words=word_tokenize(text)
    #words=[]
    
    text = ' '.join(list_of_words)
    
    
    
    return text

In [12]:
cbow_words=[]
for news in cbow_corpus.tolist():
    
    text=clean_news_cbow(news)
    cbow_words.append(text)

In [13]:
cbow_words=list(set(cbow_words))

In [14]:
news_stories=cbow_words

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(
    num_words=None, split=' '
)

tokenizer.fit_on_texts(news_stories)
print("Data fitted on the tokenizer")

Data fitted on the tokenizer


In [16]:
n_vocab = len(tokenizer.word_index.items()) + 1

In [17]:
news_sequences = tokenizer.texts_to_sequences(news_stories)
news_sequences=[x for x in news_sequences if len(x)>=2]

In [18]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(n_vocab, sampling_factor=1e-05)

print(sampling_table)

[0.00315225 0.00315225 0.00547597 ... 1.         1.         1.        ]


In [19]:
def cbow_grams(sequence, vocabulary_size,
              window_size=4, negative_samples=1., shuffle=True,
              categorical=False, sampling_table=None, seed=None):
    
    targets, contexts, labels = [], [], []    
        
    for i, wi in enumerate(sequence):
        
        
        if not wi or i < window_size or i + 1 > len(sequence)-window_size:
            continue
        if sampling_table is not None:
            if sampling_table[wi] < random.random():
                continue

        window_start = max(0, i - window_size)
        window_end = min(len(sequence), i + window_size + 1)
        
        context_words = [wj for j, wj in enumerate(sequence[window_start:window_end]) if j+window_start != i]
        target_word = wi        
            
        context_classes = tf.expand_dims(tf.constant(context_words, dtype="int64"), 0)

        negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_classes,
          num_true=window_size * 2,
          num_sampled=negative_samples,
          unique=True,
          range_max=vocabulary_size,              
          name="negative_sampling")

        # Build context and label vectors (for one target word)        
        negative_targets = negative_sampling_candidates.numpy().tolist()        

        target = [target_word] + negative_targets
        label = [1] + [0]*negative_samples

        # Append each element from the training example to global lists.
        targets.extend(target)
        contexts.extend([context_words]*(negative_samples+1))
        labels.extend(label)
        
    couples = list(zip(targets, contexts))
    
    seed = random.randint(0, 10e6)
    random.seed(seed)
    random.shuffle(couples)    
    random.seed(seed)
    random.shuffle(labels)
           
    
    return couples, labels

In [25]:
batch_size = 4096 # Data points in a single batch

embedding_size = 128 # Dimension of the embedding vector.

window_size=1 # We use a window size of 1 on either side of target word
epochs = 10 # Number of epochs to train for
negative_samples = 4 # Number of negative samples generated per example

# We pick a random validation set to sample nearest neighbors
valid_size = 16 # Random set of words to evaluate similarity on.
# We sample valid datapoints randomly from a large window without always being deterministic
valid_window = 250

# When selecting valid examples, we select some of the most frequent words as well as
# some moderately rare words as well
np.random.seed(54321)
random.seed(54321)

valid_term_ids = np.array(random.sample(range(valid_window), valid_size))
valid_term_ids = np.append(
    valid_term_ids, random.sample(range(1000, 1000+valid_window), valid_size),
    axis=0)

In [21]:
import tensorflow.keras.backend as K

K.clear_session()


# Inputs; target input layer will have the final shape [None]
# context will have [None, 2xwindow_size] shape
input_1 = tf.keras.layers.Input(shape=())
input_2 = tf.keras.layers.Input(shape=(window_size*2,))

# Target and context embedding layers
target_embedding_layer = tf.keras.layers.Embedding(
    input_dim=n_vocab, output_dim=embedding_size, name='target_embedding'
)

context_embedding_layer = tf.keras.layers.Embedding(
    input_dim=n_vocab, output_dim=embedding_size, name='context_embedding'
)

# Outputs of the target and context embedding lookups
context_out = context_embedding_layer(input_2)
target_out = target_embedding_layer(input_1)

# Taking the mean over the all the context words to produce [None, embedding_size]
mean_context_out = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=1))(context_out)

# Computing the dot product between the two 
out = tf.keras.layers.Dot(axes=-1)([context_out, target_out])

cbow_model = tf.keras.models.Model(inputs=[input_1, input_2], outputs=out, name='cbow_model')

cbow_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    optimizer='adam'
)

cbow_model.summary()

Model: "cbow_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 context_embedding (Embedding)  (None, 2, 128)       2660224     ['input_2[0][0]']                
                                                                                                  
 target_embedding (Embedding)   (None, 128)          2660224     ['input_1[0][0]']                
                                                                                         

2022-09-27 16:57:09.408876: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-27 16:57:09.408910: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-27 16:57:09.408935: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (slawomir): /proc/driver/nvidia/version does not exist
2022-09-27 16:57:09.409107: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
def cbow_data_generator(sequences, window_size, batch_size, negative_samples):
    
    rand_sequence_ids = np.arange(len(sequences))                    
    np.random.shuffle(rand_sequence_ids)

    for si in rand_sequence_ids:
        inputs, labels = cbow_grams(
            sequences[si], 
            vocabulary_size=n_vocab, 
            window_size=window_size, 
            negative_samples=negative_samples, 
            shuffle=True,
            sampling_table=sampling_table,
            seed=None
        )
        
        inputs_context, inputs_target, labels = np.array([inp[1] for inp in inputs]), np.array([inp[0] for inp in inputs]), np.array(labels).reshape(-1,1)
        
        assert inputs_context.shape[0] == inputs_target.shape[0]
        assert inputs_context.shape[0] == labels.shape[0]
        
        #print(inputs_context.shape, inputs_target.shape, labels.shape)
        for eg_id_start in range(0, inputs_context.shape[0], batch_size):            
            
            yield (
                inputs_target[eg_id_start: min(eg_id_start+batch_size, inputs_target.shape[0])], 
                inputs_context[eg_id_start: min(eg_id_start+batch_size, inputs_context.shape[0]),:]
            ), labels[eg_id_start: min(eg_id_start+batch_size, labels.shape[0])]

In [23]:
class ValidationCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, valid_term_ids, model_with_embeddings, tokenizer):
        
        self.valid_term_ids = valid_term_ids
        self.model_with_embeddings = model_with_embeddings
        self.tokenizer = tokenizer
        
        super().__init__()
        
    def on_epoch_end(self, epoch, logs=None):
        """ Validation logic """
                
        # We will use context embeddings to get the most similar words
        # Other strategies include: using target embeddings, mean embeddings after avaraging context/target
        embedding_weights = self.model_with_embeddings.get_layer("context_embedding").get_weights()[0]
        normalized_embeddings = embedding_weights / np.sqrt(np.sum(embedding_weights**2, axis=1, keepdims=True))
        
        # Get the embeddings corresponding to valid_term_ids
        valid_embeddings = normalized_embeddings[self.valid_term_ids, :]
        
        # Compute the similarity between valid_term_ids and all the embeddings
        # V x d (d x D) => V x D
        top_k = 5 # Top k items will be displayed
        similarity = np.dot(valid_embeddings, normalized_embeddings.T)
        
        # Invert similarity matrix to negative
        # Ignore the first one because that would be the same word as the probe word
        similarity_top_k = np.argsort(-similarity, axis=1)[:, 1: top_k+1]
                
        # Print the output
        for i, term_id in enumerate(valid_term_ids):
            
            similar_word_str = ', '.join([self.tokenizer.index_word[j] for j in similarity_top_k[i, :] if j >= 1])
            print(f"{self.tokenizer.index_word[term_id]}: {similar_word_str}")
        
        print('\n')

In [26]:
cbow_validation_callback = ValidationCallback(valid_term_ids, cbow_model, tokenizer)

for ei in range(epochs):
    print(f"Epoch: {ei+1}/{epochs} started")
    news_cbow_gen = cbow_data_generator(news_sequences, window_size, batch_size, negative_samples)
    cbow_model.fit(
        news_cbow_gen, 
        epochs=1, 
        callbacks=cbow_validation_callback,         
    )

Epoch: 1/10 started
  16614/Unknown - 985s 59ms/step - loss: 0.4758amount: echo, trademark, proprietary, sharp, hod
term: innovation, without, run, handlowy, cds
s: moves, maker, assumptions, pivots, gmv
stock: court, old, day, s, janw
first: second, third, fourth, inch, of
compared: collection, fo, oriola, finishing, trump
most: zm, zygmunt, bynd, transportation, primarily
investment: airlines, program, subordinate, lot, scheduled
pkn: wants, daniel, vitay, finished, secured
loss: profit, consolidated, seen, operations, income
pekao: surprising, elektrociepownia, hinting, jaston, conquered
markets: rmf, mins, prague, largely, dimon
sector: the, polish, statistical, level, gwent
pge: grupa, vcit, live, chose, stated
data: kubica, expectations, agricultural, comprise, appeared
profit: loss, result, consolidated, sales, errors
former: iii, structure, independent, branch, sells
mr: foundries, joining, stake, biib, exel
transactions: appeared, oct, kazakhstan, caution, silicon
receive: dgl

Epoch: 6/10 started
  16621/Unknown - 1015s 61ms/step - loss: 0.2974amount: number, nominal, saturday, burner, edges
term: sized, raids, esm, sabbatical, lasting
s: armed, megafon, elisa, maggie, iphi
stock: dataswarm, infoengine, fintech, eols, dance
first: second, fourth, third, framework, biscuit
compared: eur, nearly, sek, pct, honka
most: step, equally, epicenter, iko, exercize
investment: dkr, pfr, di, portfolios, society
pkn: daniel, wants, pgniig, submitted, patrycja
loss: profit, result, taxes, meur, income
pekao: pkobp, mbank, polski, pko, pzu
markets: indexes, scenario, curve, exchanges, errors
sector: agri, leader, electronic, featured, conventional
pge: grupa, mied, pknnorlen, lpp, pgnig
data: published, communication, reading, screener, environment
profit: loss, consolidated, result, taxes, income
former: pth, doering, authorized, stoen, caixabank
mr: tuomainen, katzman, helander, viitaniemi, niklas
transactions: multienerge, mazowieckie, caution, negotiate, switch
receiv

In [27]:
def save_embeddings(model, tokenizer, vocab_size, save_dir):
    
    os.makedirs(save_dir, exist_ok=True)
    
    _, words_sorted = zip(*sorted(list(tokenizer.index_word.items()), key=lambda x: x[0])[:vocab_size-1])
        
    words_sorted = [None] + list(words_sorted)
    
    pd.DataFrame(
        model.get_layer("context_embedding").get_weights()[0], 
        index = words_sorted
    ).to_pickle(os.path.join(save_dir, "context_embedding_cbow.pkl"))
    
    pd.DataFrame(
        model.get_layer("target_embedding").get_weights()[0], 
        index = words_sorted
    ).to_pickle(os.path.join(save_dir, "target_embedding_cbiw.pkl"))


In [28]:
save_embeddings(cbow_model, tokenizer, n_vocab, save_dir='cbow_embeddings')